In [ ]:
# İlk önce kullnacağımız Github reposunu indiriyoruz. Bunu sadece bir defa yapmamız yeterli.
# Yani başka zaman yapmaya gerek yok. Tabi indirilen klasörü silmezseniz
!git clone https://github.com/WongKinYiu/yolov7

In [ ]:
# Burada  modeli yükleyeceğiz.
# O yüzden dosya dizinini modelin bulunduğu klasör olarak değiştiriyoruz.
%cd yolov7

In [ ]:
#  Kullnacağımızn modelin ağırlıklarını inidiriyoruz. 
# Bu da tek seferlik bir işlem
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

In [ ]:
# Ağırlıkları yükleyerek modeli tanımlıyoruz.
from  hubconf import custom
model = custom(path_or_model='yolov7.pt')

In [ ]:
# etiketleri tanımlıyoruz.
labels=[ 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
         'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
         'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
         'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
         'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
         'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
         'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
         'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
         'hair drier', 'toothbrush' ]

In [ ]:
%cd ..

In [ ]:
# Kütüphaneleri yüklüyoruz.
import cv2
import numpy as np

In [ ]:
kamera= cv2.VideoCapture('test3.mp4')
font = cv2.FONT_HERSHEY_COMPLEX

# Kullanacağımız bölgeleri tanımlıyoruz. 
# Bunlar şeklin köşe noktaları
region1=np.array([(100,200),(400,1050),(1200,1000),(750,200)])
region1 = region1.reshape((-1,1,2))

region2=np.array([(1300,400),(1100,600),(1300,800),(1700,800),(1900,600),(1700,400)])
region2 = region2.reshape((-1,1,2))
while True:


    ret,kare=kamera.read()
    if not ret:
        break
    # Burada her bir bölge için bir değişken tanımladık.
    # Burada anlık bölgeye giren kişileri bulacağımız için
    # Her yeni görüntü alındığında bunlar sıfırlanıyor.
    total1=0
    total2=0
    
    imgs=cv2.cvtColor(kare,cv2.COLOR_BGR2RGB)
    # Resmi modele verdik
    results = model(imgs) 
    
    # Burada kça tane nesne varsa o kadar dönecek bu for döngüsü.
    # Burada her bir i ile bir nesneye bakıyoruz.
    for i in range(len(results.xyxy[0])):
        x1,y1,x2,y2,score,label=results.xyxy[0][i]
        x1,y1,x2,y2,score,label=int(x1),int(y1),int(x2),int(y2),float(score),int(label)
        name=labels[label]
        # Burada eğer nesneni score'u %20'den küçükse dikkate alınmıyor.
        # Ayrıca insan dışındaki diğer nesneler de dikkate alınmıyor.
        if score<0.2 or name!='person':
            continue
        # İnsanların orta noktasını buluyoruz.
        cx=int((x1+x2)/2)
        cy=int((y1+y2)/2)
        # orta noktaları ekranda daire olarak gösteriyoruz.
        cv2.circle(kare,(cx,cy),15,(0,255,255),-1)
        
        # Burada insanların orta noktası 1. Bölgeye girmiş mi diye bakıyoruz.
        # Eğer girmişse bu değer 1 olarak dönüyor.
        # Tam sınırdaysa 0 girmemişse -1 olarak dönüyor.
        inside_region1=cv2.pointPolygonTest(region1,(cx,cy),False)
        if inside_region1>0:
            # eğer nesne bu bölgeye girmişse o bölgeye ait değişkenin değeri 1 artıyor.
            total1+=1
            
        inside_region2=cv2.pointPolygonTest(region2,(cx,cy),False)
        if inside_region2>0:
            total2+=1
        
        
        # Burada her bir nesneye bir dikdörtgen çiziliyor.
        cv2.rectangle(kare,(x1,y1),(x2,y2),(255,0,0),2)
        # ekranda hangi nesne olduğunu ve yüzdelik score  değerini göstermek için bu değişkeni tanımlıyoruz.
        text= name+' '+str(format(score, '.2f'))
        # ekrana nesnenin adını ve yüzdelik score  değerini yazdırıyoruz
        cv2.putText(kare, text,(x1, y1-10), font, 1.5, (102,0,153), 2)
        
    # Bölgelere giren kişi sayılarını bölgelerin biraz üstüne yazdırıyoruz.
    cv2.putText(kare, str(total1),(100, 150), font, 5, (0,0,255), 5)
    cv2.putText(kare, str(total2),(1400, 400), font, 5, (255,255,0), 5)
    
    # Bölgeleri ekranda gösteriyoruz.
    cv2.polylines(kare,[region1],True,(0,255,0),10)
    cv2.polylines(kare,[region2],True,(0,150,255),10)
    # videonun çözünürlüğü büyük olduğu için ekranda gösterilirken ekrana sğısın diye boyutunu küçülttük
    kare=cv2.resize(kare,(0,0),fx=0.5,fy=0.5)
    cv2.imshow("kamera",kare)
   
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        

kamera.release()
cv2.destroyAllWindows()